In [70]:
import yfinance as yf
import pandas as pd
import os

tickers = ("AAPL", "ABBV", "ABT", "ACN", "ADBE", "AIG", "AMD", "AMGN", "AMT", "AMZN",
    "AVGO", "AXP", "BA", "BAC", "BK", "BKNG", "BLK", "BMY", "BRK-B",
    "C", "CAT", "CHTR", "CL", "CMCSA", "COF", "COP", "COST", "CRM", "CSCO",
    "CVS", "CVX", "DE", "DHR", "DIS", "DUK", "EMR", "FDX", "GD",
    "GE", "GILD", "GM", "GOOGL", "GS", "HD", "HON", "IBM", "INTC", "INTU",
    "ISRG","JNJ", "JPM", "KO", "LIN", "LLY", "LMT", "LOW", "MA", "MCD",
    "MDLZ", "MDT", "MET", "META", "MMM", "MO", "MRK", "MS", "MSFT", "NEE", "NFLX", "NKE", "NOW",
    "NVDA", "ORCL", "PEP", "PFE", "PG","PLTR", "PM", "PYPL", "QCOM", "RTX", "SBUX",
    "SCHW", "SO", "SPG", "T", "TGT", "TMO", "TMUS", "TSLA", "TXN", "UNH", "UNP", "UPS", "USB",
    "V", "VZ", "WFC", "WMT", "XOM")

In [71]:
# Import 10 years of historic data to store in data/raw using yfinance to fetch OHLCV (Open, High, Low, Close, Volume) data

path = os.path.join("..", "data", "raw")
clean_path = os.path.join("..", "data", "processed")

import_data = yf.download(tickers, start = "2015-01-01", end = "2024-12-31", auto_adjust = False)
import_data.to_csv(os.path.join(path, "import_data.csv"))

for ticker in tickers:
    dataframe = import_data.xs(ticker, level = 1, axis = 1)
    dataframe.to_csv(os.path.join(path, f"{ticker}.csv"))

[*********************100%***********************]  100 of 100 completed


In [72]:
# Pre-processing data, handle missing values/non-trading days

for file in os.listdir(path):
    file_path = os.path.join(path, file)
    clean_data = pd.read_csv(file_path, low_memory = False)
    clean_data.ffill(inplace = True)
    clean_data.bfill(inplace = True)  
    clean_data.to_csv(os.path.join(clean_path, file), index = False)

In [73]:
# Save cleaned data to /data/processed/

for file in os.listdir(clean_path):
    clean_file_path = os.path.join(clean_path, file)
    if os.path.isdir(clean_file_path):
        continue
    
    new_data = pd.read_csv(clean_file_path, low_memory = False)
    new_data['Adj Close'] = pd.to_numeric(new_data['Adj Close'], errors = 'coerce')
    new_data = new_data.drop(columns=['Open', 'High', 'Low', 'Close', 'Volume'])

    new_data.to_csv(clean_file_path, index = False) 
    final_path = os.path.join("..", "data", "processed", file)
    new_data.to_csv(final_path, index = False)  

In [74]:
# Create the return column

tickers = [ticker for ticker in os.listdir(clean_path) if ticker.endswith('.csv')]

for ticker in tickers:
    file_path = os.path.join(clean_path, ticker)
    data = pd.read_csv(file_path, low_memory = False)
    
    data["Return"] = data['Adj Close'].pct_change()
    data.to_csv(file_path, index = False)